<a href="https://colab.research.google.com/github/maufonseca18/Challenge_TelecomX_alura/blob/main/Challenge_telecom_x_alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import de los datos para el df

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
datos = pd.read_json('/content/drive/MyDrive/Proyecto telecom X Alura /TelecomX_Data.json')

In [ ]:
datos.shape

(7267, 6)

In [ ]:
datos.columns

Index(['customerID', 'Churn', 'customer', 'phone', 'internet', 'account'], dtype='object')

#Comprobación de incoherencias y arreglo

In [ ]:
columnas_dict = ['customer', 'phone', 'internet', 'account'] #Se expanden las columnas a diccionario
for col in columnas_dict:
    if col in datos.columns:
        expandido = pd.json_normalize(datos[col])
        expandido = expandido.add_prefix(f"{col}_")
        datos = pd.concat([datos.drop(columns=[col]), expandido], axis=1)

print("\nValores nulos iniciales por columna:") #Revisión inicial de nulos
print(datos.isnull().sum())


Valores nulos iniciales por columna:
customerID                   0
Churn                        0
customer_gender              0
customer_SeniorCitizen       0
customer_Partner             0
customer_Dependents          0
customer_tenure              0
phone_PhoneService           0
phone_MultipleLines          0
internet_InternetService     0
internet_OnlineSecurity      0
internet_OnlineBackup        0
internet_DeviceProtection    0
internet_TechSupport         0
internet_StreamingTV         0
internet_StreamingMovies     0
account_Contract             0
account_PaperlessBilling     0
account_PaymentMethod        0
account_Charges.Monthly      0
account_Charges.Total        0
dtype: int64


In [ ]:
columnas_objeto = datos.select_dtypes(include='object').columns #Se ajustan espacios
for col in columnas_objeto:
    datos[col] = datos[col].astype(str).str.strip().str.title()

datos['Churn'] = datos['Churn'].astype(str).str.strip() #Problema 'Churn..'#mapeo a binario
for col in columnas_objeto:
    datos[col] = datos[col].astype(str).str.strip().str.title()

datos['account_Charges.Total'] = pd.to_numeric(datos['account_Charges.Total'], errors='coerce')

In [ ]:
datos['Churn']
#Este es el cuarto intento para procesar los datos de la columna 'Churn' a binario
#Todos los demás intentos daban error y eliminaba o convertia a NaN los datos
#Trabajarlo por solitario resulve el problema....
#Es la cuarta vez que comienzo todo el code desde cero....

,Churn
0,0
1,0
2,1
3,1
4,1
...,...
7262,0
7263,1
7264,0
7265,0


In [ ]:
datos.head()

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges.Monthly,account_Charges.Total,Cuentas_Diarias
0,0002-Orfbo,0,Female,0,0,0,9,0,0,Dsl,...,0,0,0,0,One Year,0,Mailed Check,65.6,593.30,2.186667
1,0003-Mknfe,0,Male,0,0,0,9,0,0,Dsl,...,0,0,0,0,Month-To-Month,0,Mailed Check,59.9,542.40,1.996667
2,0004-Tlhlj,1,Male,0,0,0,4,0,0,Fiber Optic,...,0,0,0,0,Month-To-Month,0,Electronic Check,73.9,280.85,2.463333
3,0011-Igkff,1,Male,1,0,0,13,0,0,Fiber Optic,...,0,0,0,0,Month-To-Month,0,Electronic Check,98.0,1237.85,3.266667
4,0013-Exchz,1,Female,1,0,0,3,0,0,Fiber Optic,...,0,0,0,0,Month-To-Month,0,Mailed Check,83.9,267.40,2.796667


Estandarización y Tranformación de datos

In [ ]:
#Columna Cuentas Díarias
datos['Cuentas_Diarias'] = datos['account_Charges.Monthly'] / 30

#Columnas binarias Y/N - 1/0
cols_yes_no = [
    'customer_Partner', 'customer_Dependents',
    'phone_PhoneService', 'phone_MultipleLines',
    'internet_OnlineSecurity', 'internet_OnlineBackup',
    'internet_DeviceProtection', 'internet_TechSupport',
    'internet_StreamingTV', 'internet_StreamingMovies',
    'account_PaperlessBilling','Churn'
]

map_binario = {'yes': 1, 'no': 0}

for col in cols_yes_no:
  if col in datos.columns:
    datos[col] = datos[col].astype(str).str.strip().str.lower()
    datos[col] = datos[col].replace(['no internet service', 'no phone service'], 'no')
    datos[col] = datos[col].map(map_binario).fillna(0).astype(int)

#Verificación:
print("\nValores nulos finales por columna:")
print(datos.isnull().sum())
print("\nTipos de datos finales:")
print(datos.dtypes)
print("\nValores únicos de Churn:", datos['Churn'].unique())
print("Dimensiones finales del dataframe:", datos.shape)


Valores nulos finales por columna:
customerID                    0
Churn                         0
customer_gender               0
customer_SeniorCitizen        0
customer_Partner              0
customer_Dependents           0
customer_tenure               0
phone_PhoneService            0
phone_MultipleLines           0
internet_InternetService      0
internet_OnlineSecurity       0
internet_OnlineBackup         0
internet_DeviceProtection     0
internet_TechSupport          0
internet_StreamingTV          0
internet_StreamingMovies      0
account_Contract              0
account_PaperlessBilling      0
account_PaymentMethod         0
account_Charges.Monthly       0
account_Charges.Total        11
Cuentas_Diarias               0
dtype: int64

Tipos de datos finales:
customerID                    object
Churn                          int64
customer_gender               object
customer_SeniorCitizen         int64
customer_Partner               int64
customer_Dependents            int64


In [ ]:
# Filtrar filas con valores nulos en 'account_Charges.Total'
nulos_charges_total = datos[datos['account_Charges.Total'].isnull()]

# Mostrar cuántos son y las filas afectadas
print(f"Total de valores nulos en 'account_Charges.Total': {len(nulos_charges_total)}")
display(nulos_charges_total)

Total de valores nulos en 'account_Charges.Total': 11


,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges.Monthly,account_Charges.Total,Cuentas_Diarias
975,1371-Dwpaz,0,Female,0,0,0,0,0,0,Dsl,...,0,0,0,0,Two Year,0,Credit Card (Automatic),56.05,NaN,1.868333
1775,2520-Sgtta,0,Female,0,0,0,0,0,0,No,...,0,0,0,0,Two Year,0,Mailed Check,20.00,NaN,0.666667
1955,2775-Sefee,0,Male,0,0,0,0,0,0,Dsl,...,0,0,0,0,Two Year,0,Bank Transfer (Automatic),61.90,NaN,2.063333
2075,2923-Arzlg,0,Male,0,0,0,0,0,0,No,...,0,0,0,0,One Year,0,Mailed Check,19.70,NaN,0.656667
2232,3115-Czmzd,0,Male,0,0,0,0,0,0,No,...,0,0,0,0,Two Year,0,Mailed Check,20.25,NaN,0.675000
2308,3213-Vvolg,0,Male,0,0,0,0,0,0,No,...,0,0,0,0,Two Year,0,Mailed Check,25.35,NaN,0.845000
2930,4075-Wkniu,0,Female,0,0,0,0,0,0,Dsl,...,0,0,0,0,Two Year,0,Mailed Check,73.35,NaN,2.445000
3134,4367-Nuyao,0,Male,0,0,0,0,0,0,No,...,0,0,0,0,Two Year,0,Mailed Check,25.75,NaN,0.858333
3203,4472-Lvygi,0,Female,0,0,0,0,0,0,Dsl,...,0,0,0,0,Two Year,0,Bank Transfer (Automatic),52.55,NaN,1.751667
4169,5709-Lvoeq,0,Female,0,0,0,0,0,0,Dsl,...,0,0,0,0,Two Year,0,Mailed Check,80.85,NaN,2.695000


In [ ]:
#Cambio de los valores NaN de 'account_Charges.Total' a 0
datos.loc[
    (datos['customer_tenure'] == 0) & (datos['account_Charges.Total'].isna()),
    'account_Charges.Total'
] = 0 #Si la tendencia del cliente es 0 los datos en 'account_Charges.Total' es 0

In [ ]:
print(datos.isnull().sum()) #resulto ;)

customerID                   0
Churn                        0
customer_gender              0
customer_SeniorCitizen       0
customer_Partner             0
customer_Dependents          0
customer_tenure              0
phone_PhoneService           0
phone_MultipleLines          0
internet_InternetService     0
internet_OnlineSecurity      0
internet_OnlineBackup        0
internet_DeviceProtection    0
internet_TechSupport         0
internet_StreamingTV         0
internet_StreamingMovies     0
account_Contract             0
account_PaperlessBilling     0
account_PaymentMethod        0
account_Charges.Monthly      0
account_Charges.Total        0
Cuentas_Diarias              0
dtype: int64


#Análisis Descriptivo